# 0. Introduction

### 0.1 Description

The MNIST dataset is coming again. This time, the data contains information what `fashion` looks like. We are going to classify things like: T-Shirts, boots etc (The full list is below and inside the code...)

### 0.2 What does that data looks for humans...

In [69]:
%%html
<ul style = "list-style-type: none;">
    <li style = "float:left;"><img src ="./assets/1.png" width = 220px /></li>
    <li style = "float:left"><img src ="./assets/2.png" width = 220px /></li>
    <li style = "float:left"><img src ="./assets/3.png" width = 220px /></li>
    <li style = "float:left"><img src ="./assets/4.png" width = 220px /></li>

</ul>

### 0.3 Possible object labels

0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot 

# 1. Imports of modules

In [21]:
"""Well-known library for uploading data & preprocessing"""
import pandas as pd

"""Visualising the data"""
import matplotlib.pyplot as plt


"""ML backend"""
import tensorflow as tf
import tensorflowjs as tfjs

from sklearn.model_selection import train_test_split

"""Keras dependencies for building model"""
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 

# 2. Preprocessing

### 2.1 Reading a dataset

In [78]:
dataframe = pd.read_csv("./data/fashion-mnist_train.csv") # 28x28 images, 784 pixels
dataframe_test = pd.read_csv("./data/fashion-mnist_test.csv")  # Same like above...

In [10]:
X = dataframe.iloc[:, 1:].values # Data to train & validate during training
Y = dataframe['label'].values #Labels

### 2.2 Scaling data/ Normalisation [0 - 1] range & Reshaping

In [11]:
X = X / 255
X = X.reshape(-1,28,28,1)
Y = to_categorical(Y, 10)

In [12]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.1)

### 2.3 Augmentation

In [13]:
data_generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip= False,
    vertical_flip = False
)

data_generator.fit(X_train)

# 3. Building a model

### 3.1 Layers...

In [14]:
model = Sequential([
    Conv2D(filters = 32, kernel_size = (5, 5),activation = 'relu', input_shape = (28, 28, 1)),
    Conv2D(filters = 32, kernel_size = (5, 5)),
    MaxPool2D(pool_size = (2, 2)),
    Dropout(0.2),
    Conv2D(filters = 64, kernel_size = (3, 3),activation = 'relu'),
    Conv2D(filters = 64, kernel_size = (3, 3),activation = 'relu'),
    MaxPool2D(pool_size = (2, 2), strides = (2, 2)),
    Dropout(0.15),
    Flatten(),
    Dense(units = 256, activation = 'relu'),
    Dense (units = 10, activation = 'softmax')
    
])

### 3.2 Compilation, choosing loss function, optimizers and other parameters...

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [16]:
model.fit_generator(
    data_generator.flow(X_train, Y_train, batch_size = 128),
    epochs = 10,
    steps_per_epoch = X_train.shape[0] // 128,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/10
421/421 [==============================] - 180s 428ms/step - loss: 0.9330 - acc: 0.6483 - val_loss: 0.6023 - val_acc: 0.7613
Epoch 2/10
421/421 [==============================] - 178s 424ms/step - loss: 0.6548 - acc: 0.7494 - val_loss: 0.5653 - val_acc: 0.7733
Epoch 3/10
421/421 [==============================] - 193s 459ms/step - loss: 0.5820 - acc: 0.7787 - val_loss: 0.5229 - val_acc: 0.8005
Epoch 4/10
421/421 [==============================] - 185s 440ms/step - loss: 0.5386 - acc: 0.7955 - val_loss: 0.4512 - val_acc: 0.8270
Epoch 5/10
421/421 [==============================] - 173s 411ms/step - loss: 0.5095 - acc: 0.8055 - val_loss: 0.4591 - val_acc: 0.8288
Epoch 6/10
421/421 [==============================] - 171s 407ms/step - loss: 0.4834 - acc: 0.8182 - val_loss: 0.4176 - val_acc: 0.8418
Epoch 7/10
421/421 [==============================] - 170s 405ms/step - loss: 0.4546 - acc: 0.8310 - val_loss: 0.3983 - val_acc: 0.8495
Epoch 8/10
421/421 [============================

### 3.3 Saving to an external file

#### 3.3.1 .h5 

In [ ]:
model.save("./model/model.h5")

#### 3.3.2 .json

In [17]:
tfjs.converters.save_keras_model(model, './model_json')